## 1. Imports and Configuration

This cell handles all necessary imports and loads configuration from environment variables.

In [5]:
import os
import math
from tqdm import tqdm
from typing import List, Optional, Union, Dict

# Haystack Imports
from haystack import Document, Pipeline
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from haystack_integrations.components.embedders.ollama import OllamaDocumentEmbedder, OllamaTextEmbedder
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever

# Database Imports
from mysql.connector import connect, Error
from mysql.connector.connection import MySQLConnection
from mysql.connector.cursor import MySQLCursor, MySQLCursorAbstract
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector.abstracts import MySQLConnectionAbstract

# Load environment variables for database connection
MYSQL_HOST = os.getenv("MYSQL_HOST", "localhost")
MYSQL_PORT = int(os.getenv("MYSQL_PORT", 3306))
MYSQL_USER = os.getenv("MYSQL_USER", "root")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD", "root")
MYSQL_DB = os.getenv("MYSQL_DB", "jogos_tabuleiro")

# Configuration for ChromaDB and Ollama
CHROMA_PERSIST_PATH = "./test_databases/chroma_db_boardgames_with_popularity_scores"
CHROMA_COLLECTION_NAME = "boardgames_with_popularity_scores"
OLLAMA_MODEL = "nomic-embed-text"
OLLAMA_URL = "http://localhost:11434"

## 2. Core Functions

These functions handle database interactions, popularity calculation, and data transformation.

In [6]:
def get_mysql_connection() -> MySQLConnectionAbstract:
    """Establishes a connection to the MySQL database."""
    print(f"Connecting to MySQL: host={MYSQL_HOST}, port={MYSQL_PORT}, user={MYSQL_USER}, db={MYSQL_DB}")
    return connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=MYSQL_DB,
        port=MYSQL_PORT
    )

def fetch_boardgames_from_mysql(conn: MySQLConnectionAbstract) -> list[dict]:
    """Fetches detailed board game data from the MySQL database."""
    print("Fetching games from MySQL...")
    cursor: MySQLCursorAbstract = conn.cursor(dictionary=True)
    
    query = """
        SELECT
            jd.id, jd.nm_jogo, jd.idade_minima, jd.qt_favorito, 
            jd.qt_jogadores_max, jd.qt_jogadores_min, jd.qt_jogou, jd.qt_quer, 
            jd.qt_tem, jd.qt_teve, jd.thumb, jd.tp_jogo, jd.vl_tempo_jogo,
            GROUP_CONCAT(DISTINCT c.nm_categoria SEPARATOR ', ') as categorias,
            GROUP_CONCAT(DISTINCT t.nm_tema SEPARATOR ', ') as temas,
            GROUP_CONCAT(DISTINCT a.nm_profissional SEPARATOR ', ') as artistas,
            GROUP_CONCAT(DISTINCT m.nm_mecanica SEPARATOR ', ') as mecanicas
        FROM jogo_detalhado jd
        LEFT JOIN jogo_categoria jc ON jc.jogo_id = jd.id
        LEFT JOIN categoria c ON c.id = jc.categoria_id
        LEFT JOIN jogo_tema jt ON jt.jogo_id = jd.id
        LEFT JOIN tema t ON jt.tema_id = t.id
        LEFT JOIN jogo_artista ja ON ja.jogo_id = jd.id
        LEFT JOIN artista a ON a.id = ja.artista_id
        LEFT JOIN jogo_mecanica jm ON jm.jogo_id = jd.id
        LEFT JOIN mecanica m ON jm.mecanica_id = m.id
        WHERE jd.id IS NOT NULL AND jd.nm_jogo IS NOT NULL AND jd.nm_jogo != '' AND jd.tp_jogo = 'b'
        GROUP BY jd.id
    """
    
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()
    print(f"Found {len(results)} total games in MySQL.")
    return results

def calculate_popularity_score(game: dict) -> float:
    """Calculates a popularity score for a game using a weighted logarithmic scale."""
    weights = {
        "favoritos": 0.4,
        "jogou": 0.3,
        "tem": 0.15,
        "quer": 0.1,
        "teve": -0.05
    }
    
    log_favoritos = math.log1p(game.get('qt_favorito') or 0)
    log_jogou = math.log1p(game.get('qt_jogou') or 0)
    log_tem = math.log1p(game.get('qt_tem') or 0)
    log_quer = math.log1p(game.get('qt_quer') or 0)
    log_teve = math.log1p(game.get('qt_teve') or 0)
    
    popularity_score = (
        (log_favoritos * weights["favoritos"]) +
        (log_jogou * weights["jogou"]) +
        (log_tem * weights["tem"]) +
        (log_quer * weights["quer"]) +
        (log_teve * weights["teve"])
    )
    
    return max(0, popularity_score)

def prepare_haystack_documents(boardgames_data: list[dict]) -> list[Document]:
    """Converts a list of game data dictionaries into Haystack Document objects."""
    print("Preparing Haystack Documents...")
    haystack_docs = []
    for game in tqdm(boardgames_data, desc="Converting data to Documents"):
        content_parts = [
            f"Nome do jogo: {game.get('nm_jogo', 'N/A')}.",
            f"Tipo: {game.get('tp_jogo', 'N/A')}.",
            f"Adequado para maiores de {game.get('idade_minima', 'N/A')} anos.",
            f"Pode ser jogado por {game.get('qt_jogadores_min', 'N/A')} a {game.get('qt_jogadores_max', 'N/A')} jogadores.",
            f"Tempo médio de jogo: {game.get('vl_tempo_jogo', 'N/A')} minutos.",
            f"Descrição: {game.get('descricao', 'N/A')}." if game.get('descricao') else "Descrição não disponível.",
            f"O jogo se baseia nas seguintes categorias: {game.get('categorias', 'N/A')}." if game.get('categorias') else "",
            f"O jogo aborda os seguintes temas: {game.get('temas', 'N/A')}." if game.get('temas') else "",
            f"O jogo utiliza as seguintes mecânicas: {game.get('mecanicas', 'N/A')}." if game.get('mecanicas') else "",
            f"Artistas: {game.get('artistas')}." if game.get('artistas') else ""
        ]
        content = " ".join(filter(None, content_parts))
        
        meta = {
            "mysql_id": game.get('id'),
            "title": game.get('nm_jogo'),
            "min_age": game.get('idade_minima'),
            "max_players": game.get('qt_jogadores_max'),
            "min_players": game.get('qt_jogadores_min'),
            "play_time_minutes": game.get('vl_tempo_jogo'),
            "thumbnail": game.get('thumb'),
            "game_type": game.get('tp_jogo'),
            "categories": game.get('categorias'),
            "themes": game.get('temas'),
            "mechanics": game.get('mecanicas'),
            "artists": game.get('artistas'),
            "favorite_count": game.get('qt_favorito'),
            "played_count": game.get('qt_jogou'),
            "want_count": game.get('qt_quer'),
            "have_count": game.get('qt_tem'),
            "had_count": game.get('qt_teve'),
            "popularity_score": calculate_popularity_score(game)
        }
        
        # Remove keys with None values to keep the metadata clean
        cleaned_meta = {k: v for k, v in meta.items() if v is not None}
        
        haystack_docs.append(Document(content=content, meta=cleaned_meta))
    
    print(f"{len(haystack_docs)} Haystack Documents prepared.")
    return haystack_docs

## 3. Fetch, Prepare, and Validate Data

Execute the functions to get data from the database and transform it. This is done only once.

In [7]:
haystack_docs = []
try:
    mysql_conn = get_mysql_connection()
    boardgames_data = fetch_boardgames_from_mysql(mysql_conn)
    if boardgames_data:
        haystack_docs = prepare_haystack_documents(boardgames_data)
finally:
    if 'mysql_conn' in locals() and mysql_conn.is_connected():
        mysql_conn.close()
        print("MySQL connection closed.")

if haystack_docs:
    print("\nSample of the first Haystack Document:")
    print(f"Content: {haystack_docs[0].content[:500]}...")
    print(f"Meta: {haystack_docs[0].meta}")
else:
    print("\nNo documents were prepared. Halting script.")

Connecting to MySQL: host=localhost, port=3306, user=root, db=jogos_tabuleiro
Fetching games from MySQL...
Found 35401 total games in MySQL.
Preparing Haystack Documents...


Converting data to Documents: 100%|██████████| 35401/35401 [00:01<00:00, 35148.49it/s]

35401 Haystack Documents prepared.
MySQL connection closed.

Sample of the first Haystack Document:
Content: Nome do jogo: Old Dragon 2ª Edição: Livro I - Regras Básicas. Tipo: b. Adequado para maiores de 12 anos. Pode ser jogado por 1 a 10 jogadores. Tempo médio de jogo: 120 minutos. Descrição não disponível. O jogo aborda os seguintes temas: Aventura, Fantasia, Literatura, Medieval. O jogo utiliza as seguintes mecânicas: Narração de Histórias, Papel e Caneta, Rolagem de Dados, RPG....
Meta: {'mysql_id': 1, 'title': 'Old Dragon 2ª Edição: Livro I - Regras Básicas', 'min_age': 12, 'max_players': 10, 'min_players': 1, 'play_time_minutes': 120, 'thumbnail': 'https://storage.googleapis.com/ludopedia-capas/43761_t.jpg', 'game_type': 'b', 'themes': 'Aventura, Fantasia, Literatura, Medieval', 'mechanics': 'Narração de Histórias, Papel e Caneta, Rolagem de Dados, RPG', 'favorite_count': 69, 'played_count': 74, 'want_count': 57, 'have_count': 204, 'had_count': 1, 'popularity_score': 4.16448296

## 4. Build and Run the Indexing Pipeline

This section initializes the vector database and the Haystack indexing pipeline. It then generates embeddings for the documents and writes them to ChromaDB.

In [8]:
if haystack_docs:
    print("Initializing ChromaDocumentStore...")
    document_store = ChromaDocumentStore(
        persist_path=CHROMA_PERSIST_PATH, 
        collection_name=CHROMA_COLLECTION_NAME
    )
    print(f"ChromaDB initialized. Path: '{CHROMA_PERSIST_PATH}', Collection: '{CHROMA_COLLECTION_NAME}'.")
    print(f"Documents currently in store: {document_store.count_documents()}")

    # Initialize Haystack components for indexing
    document_embedder = OllamaDocumentEmbedder(
        model=OLLAMA_MODEL,
        url=OLLAMA_URL
    )

    writer = DocumentWriter(
        document_store=document_store,
        policy=DuplicatePolicy.OVERWRITE
    )

    # Build the indexing pipeline
    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("embedder", document_embedder)
    indexing_pipeline.add_component("writer", writer)
    indexing_pipeline.connect("embedder.documents", "writer.documents")

    # Run the pipeline in batches
    print(f"\nStarting indexing pipeline for {len(haystack_docs)} documents...")
    batch_size = 128
    for i in tqdm(range(0, len(haystack_docs), batch_size), desc="Indexing documents in batches"):
        batch_docs = haystack_docs[i:i + batch_size]
        indexing_pipeline.run({"embedder": {"documents": batch_docs}})
    
    print("\nIndexing pipeline complete.")
    print(f"Total documents in ChromaDB: {document_store.count_documents()}")
else:
    print("Skipping indexing because no documents were prepared.")

Initializing ChromaDocumentStore...
ChromaDB initialized. Path: './test_databases/chroma_db_boardgames_with_popularity_scores', Collection: 'boardgames_with_popularity_scores'.
Documents currently in store: 0

Starting indexing pipeline for 35401 documents...


Indexing documents in batches:   0%|          | 0/277 [00:00<?, ?it/s]

Indexing documents in batches: 100%|██████████| 277/277 [3:30:19<00:00, 45.56s/it]


Indexing pipeline complete.
Total documents in ChromaDB: 35401


## 5. Test the Query Pipeline

This final section demonstrates how to use the populated database to run a semantic search query.

In [9]:
print("Initializing components for querying...")

# Point to the same document store
document_store = ChromaDocumentStore(
    persist_path=CHROMA_PERSIST_PATH, 
    collection_name=CHROMA_COLLECTION_NAME
)

if document_store.count_documents() > 0:
    text_embedder = OllamaTextEmbedder(
        model=OLLAMA_MODEL, 
        url=OLLAMA_URL
    )
    embedding_retriever = ChromaEmbeddingRetriever(document_store=document_store)

    # Build the querying pipeline
    querying_pipeline = Pipeline()
    querying_pipeline.add_component("text_embedder", text_embedder)
    querying_pipeline.add_component("embedding_retriever", embedding_retriever)
    querying_pipeline.connect("text_embedder.embedding", "embedding_retriever.query_embedding")

    # Run a test query
    query_text = "jogo de estratégia para dois jogadores com tema medieval"
    print(f"\nRunning test query: '{query_text}'")
    results = querying_pipeline.run({
        "text_embedder": {"text": query_text},
        "embedding_retriever": {"top_k": 3}
    })

    # Display results
    retrieved_docs = results["embedding_retriever"]["documents"]
    if retrieved_docs:
        print("\nQuery Results:")
        for doc in retrieved_docs:
            print(f"  Title: {doc.meta.get('title', 'N/A')}, Score: {doc.score:.4f}")
            print(f"  Content snippet: {doc.content[:200]}...")
            print(f"  Meta: {doc.meta}")
            print("\n")
    else:
        print("No results found for the query.")
else:
    print("Document store is empty. Cannot run query.")

Initializing components for querying...

Running test query: 'jogo de estratégia para dois jogadores com tema medieval'

Query Results:
  Title: Stratego Conquest, Score: 397.5431
  Content snippet: Nome do jogo: Stratego Conquest. Tipo: b. Adequado para maiores de 8 anos. Pode ser jogado por 2 a 4 jogadores. Tempo médio de jogo: 60 minutos. Descrição não disponível. O jogo aborda os seguintes te...
  Meta: {'title': 'Stratego Conquest', 'want_count': 5, 'thumbnail': 'https://storage.googleapis.com/ludopedia-capas/9077_t.jpg', 'play_time_minutes': 60, 'themes': 'Medieval', 'mysql_id': 29562, 'mechanics': 'Memória, Posicionamento Secreto', 'have_count': 1, 'min_players': 2, 'popularity_score': 0.2484906649788, 'max_players': 4, 'min_age': 8, 'had_count': 1, 'game_type': 'b'}


  Title: Feudal, Score: 397.6137
  Content snippet: Nome do jogo: Feudal. Tipo: b. Adequado para maiores de 10 anos. Pode ser jogado por 2 a 6 jogadores. Tempo médio de jogo: 90 minutos. Descrição não disponível. 